In [1]:
!pip install requests
!pip install beautifulsoup4


In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://books.toscrape.com/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

titles = soup.find_all('h3')

for title in titles:
    print(title.text.strip())


A Light in the ...
Tipping the Velvet
Soumission
Sharp Objects
Sapiens: A Brief History ...
The Requiem Red
The Dirty Little Secrets ...
The Coming Woman: A ...
The Boys in the ...
The Black Maria
Starving Hearts (Triangular Trade ...
Shakespeare's Sonnets
Set Me Free
Scott Pilgrim's Precious Little ...
Rip it Up and ...
Our Band Could Be ...
Olio
Mesaerion: The Best Science ...
Libertarianism for Beginners
It's Only the Himalayas


In [3]:
import csv

with open('books.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title'])

    for title in titles:
        writer.writerow([title.text.strip()])


In [4]:
from google.colab import files
files.download('books.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import requests
from bs4 import BeautifulSoup

base_url = "https://books.toscrape.com/catalogue/page-{}.html"

all_titles = []

# نمر من الصفحة 1 إلى 50
for page_num in range(1, 51):
    url = base_url.format(page_num)
    response = requests.get(url)

    if response.status_code != 200:
        print(f"فشل تحميل الصفحة {page_num}")
        continue  # انتقل للصفحة التالية

    soup = BeautifulSoup(response.text, 'html.parser')

    # استخراج كل العناوين من الوسوم h3
    h3_tags = soup.find_all('h3')

    for h3 in h3_tags:
        a_tag = h3.find('a')
        title = a_tag['title'] if 'title' in a_tag.attrs else a_tag.text
        all_titles.append(title)

    print(f"✔ تم استخراج العناوين من الصفحة {page_num}")

# طباعة عدد العناوين المستخرجة
print(f"\n تم استخراج {len(all_titles)} عنوانًا من 50 صفحة.\n")

# طباعة بعض العناوين كمثال
for title in all_titles[:10]:
    print("-", title)


✔ تم استخراج العناوين من الصفحة 1
✔ تم استخراج العناوين من الصفحة 2
✔ تم استخراج العناوين من الصفحة 3
✔ تم استخراج العناوين من الصفحة 4
✔ تم استخراج العناوين من الصفحة 5
✔ تم استخراج العناوين من الصفحة 6
✔ تم استخراج العناوين من الصفحة 7
✔ تم استخراج العناوين من الصفحة 8
✔ تم استخراج العناوين من الصفحة 9
✔ تم استخراج العناوين من الصفحة 10
✔ تم استخراج العناوين من الصفحة 11
✔ تم استخراج العناوين من الصفحة 12
✔ تم استخراج العناوين من الصفحة 13
✔ تم استخراج العناوين من الصفحة 14
✔ تم استخراج العناوين من الصفحة 15
✔ تم استخراج العناوين من الصفحة 16
✔ تم استخراج العناوين من الصفحة 17
✔ تم استخراج العناوين من الصفحة 18
✔ تم استخراج العناوين من الصفحة 19
✔ تم استخراج العناوين من الصفحة 20
✔ تم استخراج العناوين من الصفحة 21
✔ تم استخراج العناوين من الصفحة 22
✔ تم استخراج العناوين من الصفحة 23
✔ تم استخراج العناوين من الصفحة 24
✔ تم استخراج العناوين من الصفحة 25
✔ تم استخراج العناوين من الصفحة 26
✔ تم استخراج العناوين من الصفحة 27
✔ تم استخراج العناوين من الصفحة 28
✔ تم استخراج العناوين من الصف

In [6]:
import csv

with open('book_titles.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title'])
    for title in all_titles:
        writer.writerow([title])

from google.colab import files
files.download('book_titles.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# تثبيت مكتبات (في Colab إذا احتجت)
!pip install requests beautifulsoup4 --quiet

import requests
from bs4 import BeautifulSoup
import time
import csv
from urllib.parse import urljoin

base_list_url = "https://books.toscrape.com/catalogue/page-{}.html"
site_root = "https://books.toscrape.com/"

headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; MyScraper/1.0; +https://example.com/bot)'
}

all_books = []

for page_num in range(1, 51):  # من الصفحة 1 إلى 50
    list_url = base_list_url.format(page_num)
    try:
        r = requests.get(list_url, headers=headers, timeout=15)
    except Exception as e:
        print(f"خطأ في طلب الصفحة {page_num}: {e}")
        continue

    if r.status_code != 200:
        print(f"فشل تحميل الصفحة {page_num} - status: {r.status_code}")
        continue

    soup = BeautifulSoup(r.text, 'html.parser')
    articles = soup.find_all('article', class_='product_pod')

    for art in articles:
        # عنوان و رابط صفحة المنتج (غالبًا موجود داخل <h3><a ...>)
        h3 = art.find('h3')
        a = h3.find('a')
        title = a.get('title') or a.text.strip()
        relative_link = a.get('href')
        product_url = urljoin(list_url, relative_link)  # رابط كامل

        # سعر وتوفر وتقييم موجودين في نفس بطاقة المنتج (قد نأخذ من صفحة المنتج لاحقًا أيضاً)
        price_tag = art.find('p', class_='price_color')
        price = price_tag.text.strip() if price_tag else ''

        avail_tag = art.find('p', class_='instock availability')
        availability = avail_tag.text.strip() if avail_tag else ''

        # التقييم مخزن كـ class في <p class="star-rating Three">...
        rating_tag = art.find('p', class_='star-rating')
        rating = ''
        if rating_tag:
            # توجد كلمات مثل "One", "Two", "Three", "Four", "Five" في الclass
            classes = rating_tag.get('class', [])
            # نأخذ أي كلمة ليست 'star-rating'
            rating = next((c for c in classes if c != 'star-rating'), '')

        # الآن نذهب لصفحة المنتج لجلب تفاصيل إضافية
        try:
            rp = requests.get(product_url, headers=headers, timeout=15)
        except Exception as e:
            print(f"خطأ في تحميل صفحة المنتج: {product_url} -> {e}")
            continue

        if rp.status_code != 200:
            print(f"فشل تحميل صفحة المنتج {product_url} - status: {rp.status_code}")
            continue

        psoup = BeautifulSoup(rp.text, 'html.parser')

        # وصف المنتج (عادة بعد <div id="product_description"> + <p>)
        description = ''
        desc_head = psoup.find('div', id='product_description')
        if desc_head:
            desc_p = desc_head.find_next_sibling('p')
            if desc_p:
                description = desc_p.text.strip()

        # جدول المعلومات (UPC، رقم المنتج، الخ)
        upc = ''
        product_info_table = psoup.find('table', class_='table table-striped')
        if product_info_table:
            rows = product_info_table.find_all('tr')
            for row in rows:
                th = row.find('th')
                td = row.find('td')
                if th and td:
                    key = th.text.strip().lower()
                    value = td.text.strip()
                    if key == 'upc':
                        upc = value

        # اجمع كل البيانات في dict
        book = {
            'title': title,
            'product_url': product_url,
            'price': price,
            'availability': availability,
            'rating': rating,
            'upc': upc,
            'description': description,
            'source_page': list_url
        }
        all_books.append(book)

        print(f"تم جمع: {title}")
        time.sleep(1)  # تأخير بسيط بين طلبات صفحات المنتجات

    print(f"✔ انتهت صفحة {page_num}، مجموـع حالي: {len(all_books)} كتاب")
    time.sleep(1)  # تأخير بسيط بين صفحات القائمة

# حفظ النتائج في CSV
csv_filename = 'books_detailed.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['title','product_url','price','availability','rating','upc','description','source_page']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for b in all_books:
        writer.writerow(b)

print(f"\n✅ اكتمل الكشط. تم حفظ {len(all_books)} كتاب في الملف: {csv_filename}")

# في Colab: تحميل الملف إلى جهازك
try:
    from google.colab import files
    files.download(csv_filename)
except Exception:
    pass  # إذا لم تكن في Colab تجاهل


تم جمع: A Light in the Attic
تم جمع: Tipping the Velvet
تم جمع: Soumission
تم جمع: Sharp Objects
تم جمع: Sapiens: A Brief History of Humankind
تم جمع: The Requiem Red
تم جمع: The Dirty Little Secrets of Getting Your Dream Job
تم جمع: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
تم جمع: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
تم جمع: The Black Maria
تم جمع: Starving Hearts (Triangular Trade Trilogy, #1)
تم جمع: Shakespeare's Sonnets
تم جمع: Set Me Free
تم جمع: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
تم جمع: Rip it Up and Start Again
تم جمع: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
تم جمع: Olio
تم جمع: Mesaerion: The Best Science Fiction Stories 1800-1849
تم جمع: Libertarianism for Beginners
تم جمع: It's Only the Himalayas
✔ انتهت صفحة 1، مجموـع حالي: 20 كتاب
تم جمع: In Her Wake
تم جمع: How Music Works
تم جمع: Foolproof Preserving: A 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>